In [4]:

def df_from_redshift(sql, engine):
    with engine.connect().execution_options(autocommit=True) as conn:
        query = conn.execute(text(sql))
        df = pd.DataFrame(query.fetchall())
    return df

def df_from_s3(key, bucket,aws_access_key_id,aws_secret_access_key, s3_client=None, **args):
    if s3_client is None:
        s3_client = boto3.client('s3',aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key)
    obj = s3_client.get_object(Bucket=bucket, Key=key)
    df = pd.read_parquet(io.BytesIO(obj['Body'].read()), **args)
    return df

## cem-matriculados

In [ ]:
años = ['2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023','2024']

for año in años:
        print('###########################')
        print('######   AÑO: {} ######'.format(año))
        print('###########################')

        bucket_name = 'cuech-siiuec-stage0-dev'
        path = 'cem/matriculados/periodo/{}/cem_matriculados_p_{}_iter_1.parquet'.format(año,año)

        df_s3 = df_from_s3(path, bucket_name, aws_access_key_id, aws_secret_access_key)
        #df_s3.head(2)

        texto="""
        ###########################
                Matriculados por tipo institución
        ###########################
        """
        print(texto)

        df_stage = df_s3[['tipo_inst_3','mrun']].groupby(['tipo_inst_3'],as_index=False).count()
        #df_stage

        sql_cubo = '''SELECT tipo_inst_3, count(mrun) as total_cubo 
                        FROM cem.cem_mat_p_{}
                        GROUP BY tipo_inst_3
                        ORDER BY tipo_inst_3;'''.format(año)

        df_rs = df_from_redshift(sql_cubo, engine_cubo)
        #df_rs

        df_stage['total_cubo'] = df_rs['total_cubo'].values
        df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['mrun'].values

        print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])

        texto="""
        ###########################
                Matriculados por código de institución
        ###########################
        """
        print(texto)

        df_stage = df_s3[['cod_inst','mrun']].groupby(['cod_inst'],as_index=False).count()

        sql_cubo = '''SELECT codigo_institucion, count(mrun) as total_cubo 
                        FROM cem.cem_mat_p_{}
                        GROUP BY codigo_institucion
                        ORDER BY codigo_institucion;'''.format(año)

        df_rs = df_from_redshift(sql_cubo, engine_cubo)

        df_stage['total_cubo'] = df_rs['total_cubo'].values
        df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['mrun'].values

        print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])

        texto="""
        ###########################
                Matriculados por región
        ###########################
        """
        print(texto)

        df_stage = df_s3[['region_sede','mrun']].groupby(['region_sede'],as_index=False).count()

        mapeo = {"Coquimbo":"Región de Coquimbo",
                "Ñuble":"Región de Ñuble",
                "Valparaíso":"Región de Valparaíso",
                "La Araucanía":"Región de La Araucanía",
                "Tarapacá":"Región de Tarapacá",
                "Magallanes":"Región de Magallanes y Antártica Chilena",
                "Arica y Parinacota":"Región de Arica y Parinacota",
                "Los Lagos":"Región de Los Lagos",
                "Antofagasta":"Región de Antofagasta",
                "Los Ríos":"Región de Los Ríos",
                "Biobío":"Región del Bío-Bío",
                "Lib. Gral B. O'Higgins":"Región del Libertador Bernardo O'Higgins",
                "Atacama":"Región de Atacama",
                "Aysén":"Región de Aysén del Gral.Ibañez del Campo",
                "Maule":"Región del Maule",
                "Metropolitana":"Región Metropolitana de Santiago"}

        df_stage['region_sede'] = df_stage['region_sede'].map(mapeo)
        #df_stage

        sql_cubo = '''SELECT region_sede, count(mrun) as total_cubo 
                        FROM cem.cem_mat_p_{}
                        GROUP BY region_sede
                        ORDER BY region_sede;'''.format(año)

        df_rs = df_from_redshift(sql_cubo, engine_cubo)
        #df_rs

        df_stage = df_stage.sort_values(['region_sede'])
        df_rs = df_rs.sort_values(['region_sede'])

        df_stage['total_cubo'] = df_rs['total_cubo'].values
        df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['mrun'].values

        print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])

## cem-titulados

In [ ]:
años = ['2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023']

for año in años:
        print('###########################')
        print('######   AÑO: {} ######'.format(año))
        print('###########################')

        bucket_name = 'cuech-siiuec-stage0-dev'
        path = 'cem/titulados/periodo/{}/cem_titulados_p_{}_iter_1.parquet'.format(año,año)

        df_s3 = df_from_s3(path, bucket_name, aws_access_key_id, aws_secret_access_key)
        #df_s3.head(2)

        texto="""
        ###########################
                Titulados por tipo institución
        ###########################
        """
        print(texto)

        df_stage = df_s3[['tipo_inst_3','mrun']].groupby(['tipo_inst_3'],as_index=False).count()
        #df_stage

        sql_cubo = '''SELECT tipo_inst_3, count(mrun) as total_cubo 
                        FROM cem.cem_tit_p_{}
                        GROUP BY tipo_inst_3
                        ORDER BY tipo_inst_3;'''.format(año)

        df_rs = df_from_redshift(sql_cubo, engine_cubo)
        #df_rs

        df_stage['total_cubo'] = df_rs['total_cubo'].values
        df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['mrun'].values

        print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])

        texto="""
        ###########################
                Titulados por código de institución
        ###########################
        """
        print(texto)

        df_stage = df_s3[['cod_inst','mrun']].groupby(['cod_inst'],as_index=False).count()

        sql_cubo = '''SELECT codigo_institucion, count(mrun) as total_cubo 
                        FROM cem.cem_tit_p_{}
                        GROUP BY codigo_institucion
                        ORDER BY codigo_institucion;'''.format(año)

        df_rs = df_from_redshift(sql_cubo, engine_cubo)

        df_stage['total_cubo'] = df_rs['total_cubo'].values
        df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['mrun'].values

        print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])

        texto="""
        ###########################
                Titulados por región
        ###########################
        """
        print(texto)

        df_stage = df_s3[['region_sede','mrun']].groupby(['region_sede'],as_index=False).count()

        mapeo = {"Coquimbo":"Región de Coquimbo",
                "Ñuble":"Región de Ñuble",
                "Valparaíso":"Región de Valparaíso",
                "La Araucanía":"Región de La Araucanía",
                "Tarapacá":"Región de Tarapacá",
                "Magallanes":"Región de Magallanes y Antártica Chilena",
                "Arica y Parinacota":"Región de Arica y Parinacota",
                "Los Lagos":"Región de Los Lagos",
                "Antofagasta":"Región de Antofagasta",
                "Los Ríos":"Región de Los Ríos",
                "Biobío":"Región del Bío-Bío",
                "Lib. Gral B. O'Higgins":"Región del Libertador Bernardo O'Higgins",
                "Atacama":"Región de Atacama",
                "Aysén":"Región de Aysén del Gral.Ibañez del Campo",
                "Maule":"Región del Maule",
                "Metropolitana":"Región Metropolitana de Santiago"}

        df_stage['region_sede'] = df_stage['region_sede'].map(mapeo)
        #df_stage

        sql_cubo = '''SELECT region_sede, count(mrun) as total_cubo 
                        FROM cem.cem_tit_p_{}
                        GROUP BY region_sede
                        ORDER BY region_sede;'''.format(año)

        df_rs = df_from_redshift(sql_cubo, engine_cubo)
        #df_rs

        df_stage = df_stage.sort_values(['region_sede'])
        df_rs = df_rs.sort_values(['region_sede'])

        df_stage['total_cubo'] = df_rs['total_cubo'].values
        df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['mrun'].values

        print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])

## cem-afuas

In [ ]:
años = ['2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023']

for año in años:
        print('###########################')
        print('######   AÑO: {} ######'.format(año))
        print('###########################')

        bucket_name = 'cuech-siiuec-stage0-dev'
        path = 'cem/afuas/periodo/{}/cem_afuas_p_{}.parquet'.format(año,año)

        df_s3 = df_from_s3(path, bucket_name, aws_access_key_id, aws_secret_access_key)
        #df_s3.head(2)

        texto="""
        ###########################
                Postulante por beneficio_beca_fscu
        ###########################
        """
        print(texto)

        df_stage = df_s3[['beneficio_beca_fscu','mrun']].groupby(['beneficio_beca_fscu'],as_index=False).count()
        #df_stage

        sql_cubo = '''SELECT beneficio_beca_fscu, count(mrun) as total_cubo 
                        FROM cem.cem_afuas_p_{}
                        GROUP BY beneficio_beca_fscu
                        ORDER BY beneficio_beca_fscu;'''.format(año)

        df_rs = df_from_redshift(sql_cubo, engine_cubo)
        #df_rs

        df_stage['total_cubo'] = df_rs['total_cubo'].values
        df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['mrun'].values

        print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])

        texto="""
        ###########################
                Postulante por tipo_alumno
        ###########################
        """
        print(texto)

        df_stage = df_s3[['tipo_alumno','mrun']].groupby(['tipo_alumno'],as_index=False).count()

        sql_cubo = '''SELECT tipo_alumno, count(mrun) as total_cubo 
                        FROM cem.cem_afuas_p_{}
                        GROUP BY tipo_alumno
                        ORDER BY tipo_alumno;'''.format(año)

        df_rs = df_from_redshift(sql_cubo, engine_cubo)

        df_stage['total_cubo'] = df_rs['total_cubo'].values
        df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['mrun'].values

        print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])

        texto="""
        ###########################
                Postulante por quintil_ingreso
        ###########################
        """
        print(texto)

        df_stage = df_s3[['quintil_ingreso','mrun']].groupby(['quintil_ingreso'],as_index=False).count()

        sql_cubo = '''SELECT quintil_ingreso, count(mrun) as total_cubo 
                        FROM cem.cem_afuas_p_{}
                        GROUP BY quintil_ingreso
                        ORDER BY quintil_ingreso;'''.format(año)

        df_rs = df_from_redshift(sql_cubo, engine_cubo)

        df_stage = df_stage.sort_values(['quintil_ingreso'])
        df_rs = df_rs.sort_values(['quintil_ingreso'])

        df_stage['total_cubo'] = df_rs['total_cubo'].values
        df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['mrun'].values

        print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])

## cem-pfuas

In [ ]:
años = ['2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023']

for año in años:
        print('###########################')
        print('######   AÑO: {} ######'.format(año))
        print('###########################')

        bucket_name = 'cuech-siiuec-stage0-dev'
        path = 'cem/pfuas/periodo/{}/cem_pfuas_p_{}.parquet'.format(año,año)

        df_s3 = df_from_s3(path, bucket_name, aws_access_key_id, aws_secret_access_key)
        #df_s3.head(2)

        texto="""
        ###########################
                Postulante por genero
        ###########################
        """
        print(texto)

        df_stage = df_s3[['genero','mrun']].groupby(['genero'],as_index=False).count()
        #df_stage

        sql_cubo = '''SELECT genero, count(mrun) as total_cubo 
                        FROM cem.cem_pfuas_p_{}
                        GROUP BY genero
                        ORDER BY genero;'''.format(año)

        df_rs = df_from_redshift(sql_cubo, engine_cubo)
        #df_rs

        df_stage['total_cubo'] = df_rs['total_cubo'].values
        df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['mrun'].values

        print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])

        texto="""
        ###########################
                Postulante por edad
        ###########################
        """
        print(texto)

        df_stage = df_s3[['edad','mrun']].groupby(['edad'],as_index=False).count()

        sql_cubo = '''SELECT edad, count(mrun) as total_cubo 
                        FROM cem.cem_pfuas_p_{}
                        GROUP BY edad
                        ORDER BY edad;'''.format(año)

        df_rs = df_from_redshift(sql_cubo, engine_cubo)

        df_stage['total_cubo'] = df_rs['total_cubo'].values
        df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['mrun'].values

        print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])

        texto="""
        ###########################
                Postulante por proceso
        ###########################
        """
        print(texto)

        df_stage = df_s3[['proceso','mrun']].groupby(['proceso'],as_index=False).count()

        sql_cubo = '''SELECT proceso, count(mrun) as total_cubo 
                        FROM cem.cem_pfuas_p_{}
                        GROUP BY proceso
                        ORDER BY proceso;'''.format(año)

        df_rs = df_from_redshift(sql_cubo, engine_cubo)

        df_stage = df_stage.sort_values(['proceso'])
        df_rs = df_rs.sort_values(['proceso'])

        df_stage['total_cubo'] = df_rs['total_cubo'].values
        df_stage['diferencias'] = df_stage['total_cubo'].values - df_stage['mrun'].values

        print('Diferencias: \n\n', df_stage.loc[(df_stage.loc[:,'diferencias'] != 0)])